In [7]:
import pandas as pd
import numpy as np

df=pd.read_csv('master_instruments2.csv')

In [8]:
df.columns

Index(['Unnamed: 0', 'ExchangeSegment', 'ExchangeInstrumentID', 'Name',
       'Description', 'Series', 'NameWithSeries', 'InstrumentID',
       'PriceBand.High', 'PriceBand.Low', 'FreezeQty', 'TickSize', 'LotSize',
       'Multiplier', 'displayName', 'ExpiryDate', 'Strike', 'OptionType',
       'TickerName'],
      dtype='object')

In [17]:
import json
import csv

# Desired CSV columns:
# Index(['Unnamed: 0', 'ExchangeSegment', 'ExchangeInstrumentID', 'Name',
#        'Description', 'Series', 'NameWithSeries', 'InstrumentID',
#        'PriceBand.High', 'PriceBand.Low', 'FreezeQty', 'TickSize', 'LotSize',
#        'Multiplier', 'displayName', 'ExpiryDate', 'Strike', 'OptionType',
#        'TickerName'],
#       dtype='object')
#
# In our mapping from the pipe‑delimited string (with 23 fields), we use:
#
# Record fields (indices):
#  0: ExchangeSegment                -> CSV: ExchangeSegment
#  1: ExchangeInstrumentID           -> CSV: ExchangeInstrumentID
#  2: (ignored)
#  3: Name                           -> CSV: Name and displayName
#  4: Contract Symbol (e.g. CONCOR25APR780PE) -> used to extract Series & NameWithSeries
#  5: (ignored)
#  6: (ignored)
#  7: InstrumentID                   -> CSV: InstrumentID
#  8: PriceBand.High                 -> CSV: PriceBand.High
#  9: PriceBand.Low                  -> CSV: PriceBand.Low
# 10: FreezeQty                      -> CSV: FreezeQty
# 11: TickSize                       -> CSV: TickSize
# 12: LotSize                        -> CSV: LotSize
# 13: Multiplier                     -> CSV: Multiplier
# 14: (ignored)
# 15: (often same as Name)           -> (ignored, we use Name from index 3)
# 16: ExpiryDate                     -> CSV: ExpiryDate
# 17: Strike                         -> CSV: Strike
# 18: OptionType                     -> CSV: OptionType
# 19: Description (option description) -> CSV: Description
# 20: (ignored)
# 21: (ignored)
# 22: TickerName (usually same as the contract symbol) -> CSV: TickerName
#
# We also generate "Unnamed: 0" as a sequential index.

# Define header for CSV
csv_header = [
    "Unnamed: 0",
    "ExchangeSegment",
    "ExchangeInstrumentID",
    "Name",
    "Description",
 
    "NameWithSeries",
    "InstrumentID",
    "PriceBand.High",
    "PriceBand.Low",
    "FreezeQty",
    "TickSize",
    "LotSize",
    "Multiplier",
    "displayName",
    "ExpiryDate",
    "Strike",
    "OptionType",
    "TickerName"
]

def parse_record(record_str):
    """
    Splits a pipe-delimited string and returns the list of fields.
    """
    fields = record_str.split("|")
    if len(fields) < 23:
        print(f"Warning: Expected at least 23 fields, but got {len(fields)} in record: {record_str}")
    return fields

def extract_series(contract_symbol):
    """
    Extracts the option series (e.g. 'CE' or 'PE') from the contract symbol.
    Assumes the series is the last two characters.
    """
    return contract_symbol[-2:] if len(contract_symbol) >= 2 else ""

def main():
    # Read the JSON file containing the pipe-delimited strings.
    with open("master_instruments.json", "r") as f:
        data = json.load(f)
    
    csv_rows = []
    
    for idx, record_str in enumerate(data):
        fields = parse_record(record_str)
        # Ensure we have enough fields to map our data (we need index up to 22)
        if len(fields) < 23:
            continue  # skip or handle as needed

        # Map the fields to the desired CSV columns
        # Using the mapping defined in the header comments above.
        row = [
            idx,                        # Unnamed: 0 (sequential index)
            fields[0],                  # ExchangeSegment
            fields[1],                  # ExchangeInstrumentID
            fields[3],                  # Name
            fields[19],                 # Description (option description)
        
            fields[4],                  # NameWithSeries (the full contract symbol)
            fields[7],                  # InstrumentID
            fields[8],                  # PriceBand.High
            fields[9],                  # PriceBand.Low
            fields[10],                 # FreezeQty
            fields[11],                 # TickSize
            fields[12],                 # LotSize
            fields[13],                 # Multiplier
            fields[3],                  # displayName (using Name from index 3)
            fields[16],                 # ExpiryDate
            fields[17],                 # Strike
            fields[18],                 # OptionType
            fields[22]                  # TickerName (contract code)
        ]
        csv_rows.append(row)
    
    # Write the rows to CSV
    with open("data.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_header)
        writer.writerows(csv_rows)
    
    print("CSV file 'data.csv' created successfully.")

if __name__ == "__main__":
    main()


CSV file 'data.csv' created successfully.


In [2]:
df["ExpiryDate"].unique()   

array(['2025-03-27 14:30:00', '2025-04-24 14:30:00',
       '2025-02-27 14:30:00'], dtype=object)

In [21]:
import re

def extract_futures(symbol, df):
    """
    Extracts future contract details from the given symbol.

    Args:
    symbol (str): The future contract description.
    df (DataFrame): The DataFrame containing futures contract data.

    Returns:
    tuple: (ExchangeInstrumentID of first expiry, second expiry contract description)
    """
    # Regular expression to match stock symbol and expiry months
    pattern = re.match(r"([A-Z\&\-]+)(\d{2}[A-Z]+)(\d{2}[A-Z]+)FUT", symbol)
    
    if pattern:
        stock_symbol = pattern.group(1)  # Extract stock symbol (e.g., TITAN)
        expiry1 = pattern.group(2)       # First expiry (e.g., 25MAR)
        expiry2 = pattern.group(3)       # Second expiry (e.g., 25APR)

        # Check if the first future contract exists in the DataFrame
        fut1_df = df[df['Description'] == f"{stock_symbol}{expiry1}FUT"]

        if not fut1_df.empty:
            fut1_exchange_id = fut1_df['ExchangeInstrumentID'].values[0]  # Extract ExchangeInstrumentID safely
        else:
            fut1_exchange_id = None  # Handle missing case

        fut2 = f"{stock_symbol}{expiry2}FUT"

        return fut1_exchange_id, fut2
    else:
        return None

# Iterate over descriptions ending with "SPD"
for i in df[df["Strike"].str.endswith("SPD")]['Description']:
    result = extract_futures(i, df)

    if result:
        print("Extracted Futures:")
        print(result[0] if result[0] is not None else "ExchangeInstrumentID not found")  # TITAN25MARFUT ExchangeInstrumentID or None
        print(result[1])  # TITAN25APRFUT
    else:
        print(f"No match found for {i}")


Extracted Futures:
41463
TECHM25MARFUT
Extracted Futures:
43400
TITAN25APRFUT
Extracted Futures:
41029
ITC25APRFUT
Extracted Futures:
35057
ADANIENT25APRFUT
Extracted Futures:
43438
WIPRO25APRFUT
Extracted Futures:
41559
WIPRO25MARFUT
Extracted Futures:
43396
TECHM25APRFUT
Extracted Futures:
35033
ADANIENT25MARFUT
Extracted Futures:
41513
ULTRACEMCO25MARFUT
Extracted Futures:
39122
JSWSTEEL25MARFUT
Extracted Futures:
35215
ASIANPAINT25APRFUT
Extracted Futures:
39039
ITC25MARFUT
Extracted Futures:
35245
AXISBANK25APRFUT
Extracted Futures:
41615
JSWSTEEL25APRFUT
Extracted Futures:
35310
AXISBANK25MARFUT
Extracted Futures:
43406
ULTRACEMCO25APRFUT
Extracted Futures:
35154
ASIANPAINT25MARFUT
Extracted Futures:
41498
TITAN25MARFUT


In [23]:
import re
import json

def extract_futures(symbol, df):
    """
    Extracts future contract details from the given symbol.

    Args:
    symbol (str): The future contract description.
    df (DataFrame): The DataFrame containing futures contract data.

    Returns:
    tuple: (ExchangeInstrumentID of first expiry, ExchangeInstrumentID of second expiry)
    """
    # Regular expression to match stock symbol and expiry months
    pattern = re.match(r"([A-Z\&\-]+)(\d{2}[A-Z]+)(\d{2}[A-Z]+)FUT", symbol)
    
    if pattern:
        stock_symbol = pattern.group(1)  # Extract stock symbol (e.g., TITAN)
        expiry1 = pattern.group(2)       # First expiry (e.g., 25MAR)
        expiry2 = pattern.group(3)       # Second expiry (e.g., 25APR)

        # Fetch F1 ExchangeInstrumentID
        fut1_df = df[df['Description'] == f"{stock_symbol}{expiry1}FUT"]
        fut1_exchange_id = int(fut1_df['ExchangeInstrumentID'].values[0]) if not fut1_df.empty else None

        # Fetch F2 ExchangeInstrumentID
        fut2_df = df[df['Description'] == f"{stock_symbol}{expiry2}FUT"]
        fut2_exchange_id = int(fut2_df['ExchangeInstrumentID'].values[0]) if not fut2_df.empty else None

        return fut1_exchange_id, fut2_exchange_id
    else:
        return None, None

# Dictionary to store results
futures_mapping = {}

# Iterate over descriptions ending with "SPD"
for _, row in df[df["Strike"].str.endswith("SPD")].iterrows():
    parent_exchange_id = int(row['ExchangeInstrumentID'])  # Get parent ExchangeInstrumentID
    result = extract_futures(row['Description'], df)

    if result:
        f1_exchange_id = result[0]
        f2_exchange_id = result[1]
        
        # Store only if both F1 and F2 are found
        if f1_exchange_id and f2_exchange_id:
            futures_mapping[parent_exchange_id] = [f1_exchange_id, f2_exchange_id]

# Save to JSON file
output_file = r"D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json"
with open(output_file, "w") as json_file:
    json.dump(futures_mapping, json_file, indent=4)

print(f"Futures mapping saved at: {output_file}")


Futures mapping saved at: D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json


In [25]:
import re
import json

def extract_futures(symbol, df):
    """
    Extracts future contract details from the given symbol.

    Args:
    symbol (str): The future contract description.
    df (DataFrame): The DataFrame containing futures contract data.

    Returns:
    tuple: (ExchangeInstrumentID of first expiry, ExchangeInstrumentID of second expiry)
    """
    # Regular expression to match stock symbol and expiry months
    pattern = re.match(r"([A-Z\&\-]+)(\d{2}[A-Z]+)(\d{2}[A-Z]+)FUT", symbol)
    
    if pattern:
        stock_symbol = pattern.group(1)  # Extract stock symbol (e.g., TITAN)
        expiry1 = pattern.group(2)       # First expiry (e.g., 25MAR)
        expiry2 = pattern.group(3)       # Second expiry (e.g., 25APR)

        # Fetch F1 ExchangeInstrumentID
        fut1_df = df[df['Description'] == f"{stock_symbol}{expiry1}FUT"]
        fut1_exchange_id = int(fut1_df['ExchangeInstrumentID'].values[0]) if not fut1_df.empty else None

        # Fetch F2 ExchangeInstrumentID
        fut2_df = df[df['Description'] == f"{stock_symbol}{expiry2}FUT"]
        fut2_exchange_id = int(fut2_df['ExchangeInstrumentID'].values[0]) if not fut2_df.empty else None

        return fut1_exchange_id, fut2_exchange_id
    else:
        return None, None

# Dictionary to store parent-child mapping
futures_mapping = {}

# List to store ExchangeInstrumentIDs
exchange_instrument_list = []

# Iterate over descriptions ending with "SPD"
for _, row in df[df["Strike"].str.endswith("SPD")].iterrows():
    parent_exchange_id = int(row['ExchangeInstrumentID'])  # Convert to int
    result = extract_futures(row['Description'], df)

    if result:
        f1_exchange_id = result[0]
        f2_exchange_id = result[1]

        # Store only if both F1 and F2 are found
        if f1_exchange_id and f2_exchange_id:
            futures_mapping[parent_exchange_id] = [f1_exchange_id, f2_exchange_id]

            # Add all IDs to the list
            exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": f1_exchange_id})
            exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": f2_exchange_id})

    # Add parent exchangeInstrumentID to the list as well
    exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": parent_exchange_id})

# Remove duplicates from the list
exchange_instrument_list = list({v["exchangeInstrumentID"]: v for v in exchange_instrument_list}.values())

# Save Futures Mapping to JSON
futures_mapping_file = r"D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json"
with open(futures_mapping_file, "w") as json_file:
    json.dump(futures_mapping, json_file, indent=4)

# Save ExchangeInstrumentID List to JSON
exchange_instrument_file = r"D:\YASH\Cleaning\Data\Futuresdataforwardfil\exchange_instruments.json"
with open(exchange_instrument_file, "w") as json_file:
    json.dump(exchange_instrument_list, json_file, indent=4)

print(f"✅ Futures mapping saved at: {futures_mapping_file}")
print(f"✅ Exchange Instrument List saved at: {exchange_instrument_file}")


✅ Futures mapping saved at: D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json
✅ Exchange Instrument List saved at: D:\YASH\Cleaning\Data\Futuresdataforwardfil\exchange_instruments.json


In [3]:
df[df["Name"]=="TITAN"]

,Unnamed: 0,ExchangeSegment,ExchangeInstrumentID,Name,Description,Series,NameWithSeries,InstrumentID,PriceBand.High,PriceBand.Low,FreezeQty,TickSize,LotSize,Multiplier,displayName,ExpiryDate,Strike,OptionType,TickerName
2,2,NSEFO,73632,TITAN,TITAN25APRFUT,FUTSTK,TITAN-FUTSTK,2511400073632,3907.00,3196.65,7001,0.05,175,1,TITAN,2025-04-24 14:30:00,TITAN 24APR2025,1,1
5,5,NSEFO,43400,TITAN,TITAN25MARFUT,FUTSTK,TITAN-FUTSTK,2508600043400,3881.35,3175.65,7001,0.05,175,1,TITAN,2025-03-27 14:30:00,TITAN 27MAR2025,1,1
24,24,NSEFO,41498,TITAN,TITAN25FEBFUT,FUTSTK,TITAN-FUTSTK,2505800041498,3858.80,3157.20,7001,0.05,175,1,TITAN,2025-02-27 14:30:00,TITAN 27FEB2025,1,1
28,28,NSEFO,11184032,TITAN,TITAN25MAR25APRFUT,FUTSTK,TITAN-FUTSTK,2100111184032,88.25,-88.25,7001,0.05,175,1,NaN,2025-03-27 14:30:00,TITAN 27MAR24APR SPD,1,1
44,44,NSEFO,10666888,TITAN,TITAN25FEB25MARFUT,FUTSTK,TITAN-FUTSTK,2100110666888,87.70,-87.70,7001,0.05,175,1,NaN,2025-02-27 14:30:00,TITAN 27FEB27MAR SPD,1,1


In [26]:
#create a dic of ExchangeInstrumentID and  Description

df1=df[["ExchangeInstrumentID","Description"]]
df1=df1.drop_duplicates()
df1=df1.set_index("ExchangeInstrumentID")
df1=df1.to_dict()
#save to json
import json
with open('ExchangeInstrumentID.json', 'w') as fp:
    json.dump(df1, fp)
    

In [17]:
import re

def extract_symbols(text):


    matches = re.findall(r'^([A-Z]+)\s\d{2}([A-Z]+)\d{2}([A-Z]+)\sSPD$', text)
    
    if matches:

        stock_name, month1, month2 = matches[0]
        symbol1 = f"{stock_name}{month1}"
        symbol2 = f"{stock_name}{month2}"
        return symbol1, symbol2
    else:
        return None
spdsym=df[df["displayName"].isna()]["Description"].apply(extract_symbols)
print(spdsym)
result = extract_symbols(spdsym)

if result:
    print("Symbol 1:", result[0])  # Output: TITANJAN
    print("Symbol 2:", result[1])  # Output: TITANFEB
else:
    print("No valid symbols found")


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
Name: Description, dtype: object


TypeError: expected string or bytes-like object, got 'Series'

In [25]:
df["displayName"].unique()


array([nan, 'TITAN', 'JSWSTEEL', 'TECHM', 'WIPRO', 'ADANIENT',
       'ULTRACEMCO', 'ITC', 'ASIANPAINT', 'AXISBANK'], dtype=object)

In [27]:
asp=df[df["Name"]=="JSWSTEEL"]
asp

,Unnamed: 0,ExchangeSegment,ExchangeInstrumentID,Name,Description,Series,NameWithSeries,InstrumentID,PriceBand.High,PriceBand.Low,FreezeQty,TickSize,LotSize,Multiplier,displayName,ExpiryDate,Strike,OptionType,TickerName
9,9,NSEFO,10056847,JSWSTEEL,JSWSTEEL25FEB25MARFUT,FUTSTK,JSWSTEEL-FUTSTK,2100110056847,22.65,-22.65,27001,0.05,675,1,NaN,2025-02-27 14:30:00,JSWSTEEL 27FEB27MAR SPD,1,1
17,17,NSEFO,10559186,JSWSTEEL,JSWSTEEL25JAN25FEBFUT,FUTSTK,JSWSTEEL-FUTSTK,2100110559186,22.55,-22.55,27001,0.05,675,1,NaN,2025-01-30 14:30:00,JSWSTEEL 30JAN27FEB SPD,1,1
19,19,NSEFO,41615,JSWSTEEL,JSWSTEEL25MARFUT,FUTSTK,JSWSTEEL-FUTSTK,2508600041615,1003.30,820.85,27001,0.05,675,1,JSWSTEEL,2025-03-27 14:30:00,JSWSTEEL 27MAR2025,1,1
40,40,NSEFO,39122,JSWSTEEL,JSWSTEEL25FEBFUT,FUTSTK,JSWSTEEL-FUTSTK,2505800039122,997.00,815.75,27001,0.05,675,1,JSWSTEEL,2025-02-27 14:30:00,JSWSTEEL 27FEB2025,1,1
43,43,NSEFO,41094,JSWSTEEL,JSWSTEEL25JANFUT,FUTSTK,JSWSTEEL-FUTSTK,2503000041094,990.95,810.80,27001,0.05,675,1,JSWSTEEL,2025-01-30 14:30:00,JSWSTEEL 30JAN2025,1,1


In [ ]:
{10559186:[41094,39122]}

In [49]:
df["InstrumentID"].unique()
df["Name"].unique()


instrument_list = []

df = df.drop_duplicates(subset=['InstrumentID'])
asp = asp.reset_index(drop=True)
for i in range(len(asp["InstrumentID"])):

    instrument_list.append({"exchangeSegment":2,"exchangeInstrumentID":int(asp["ExchangeInstrumentID"][i])})
#save into json file
import json
with open('Titaninstrument_list.json', 'w') as f:
    json.dump(instrument_list, f)


In [51]:
df["InstrumentID"].unique()
df["Name"].unique()


instrument_list = []

df = df.drop_duplicates(subset=['InstrumentID'])
asp = asp.reset_index(drop=True)


#sort asp by expiry date
asp=asp.sort_values(by="ExpiryDate")

for i in range(len(asp["InstrumentID"])):

    instrument_list.append({"exchangeSegment":2,"exchangeInstrumentID":int(asp["ExchangeInstrumentID"][i])})
#save into json file
import json
with open('Titaninstrument_list.json', 'w') as f:
    json.dump(instrument_list, f)


In [6]:
asp.columns

NameError: name 'asp' is not defined

In [77]:
import pandas as pd

df5=pd.read_csv('formateddata.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_17636\3415216963.py:3: DtypeWarning: Columns (17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df5=pd.read_csv('formateddata.csv')


In [44]:
df5["instrument_category"].unique()

array([2, 1, 4])

In [70]:
df5

,exchange,token,instrument_category,underlying,contract_symbol,instrument_type,product,contract_unique_id,price1,price2,...,trading_status,alternate_id,underlying_name,expiry,strike,option_type,option_description,flag1,flag2,contract_code
0,NSEFO,101335,2,CONCOR,CONCOR25APR780PE,OPTSTK,CONCOR-OPTSTK,2511400101335,123.90,63.30,...,1,1100100004749,CONCOR,2025-04-24T14:30:00,780.0,4,CONCOR 24APR2025 PE 780,1,1.0,CONCOR25APR780PE
1,NSEFO,75867,2,BANKNIFTY,BANKNIFTY25APR66000CE,OPTIDX,BANKNIFTY-OPTIDX,2511400075867,21.35,0.05,...,1,-1,Nifty Bank,2025-04-24T14:30:00,66000.0,3,BANKNIFTY 24APR2025 CE 66000,1,1.0,BANKNIFTY25APR66000CE
2,NSEFO,102204,2,DELHIVERY,DELHIVERY25FEB245CE,OPTSTK,DELHIVERY-OPTSTK,2505800102204,49.80,9.80,...,1,1100100009599,DELHIVERY,2025-02-27T14:30:00,245.0,3,DELHIVERY 27FEB2025 CE 245,1,1.0,DELHIVERY25FEB245CE
3,NSEFO,78442,2,AUBANK,AUBANK25MAR530CE,OPTSTK,AUBANK-OPTSTK,2508600078442,61.40,17.50,...,1,1100100021238,AUBANK,2025-03-27T14:30:00,530.0,3,AUBANK 27MAR2025 CE 530,1,1.0,AUBANK25MAR530CE
4,NSEFO,134839,2,NHPC,NHPC25MAR60CE,OPTSTK,NHPC-OPTSTK,2508600134839,34.75,0.05,...,1,1100100017400,NHPC,2025-03-27T14:30:00,60.0,3,NHPC 27MAR2025 CE 60,1,1.0,NHPC25MAR60CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86507,NSEFO,10436573,4,IRB,IRB25MAR25APRFUT,FUTSTK,IRB-FUTSTK,2100110436573,1.25,-1.25,...,1,-1,NaN,2025-03-27T14:30:00,IRB 27MAR24APR SPD,1,1,IRB25MAR25APRFUT,NaN,NaN
86508,NSEFO,9388895,4,FEDERALBNK,FEDERALBNK25MAR25APRFUT,FUTSTK,FEDERALBNK-FUTSTK,2100109388895,4.62,-4.62,...,1,-1,NaN,2025-03-27T14:30:00,FEDERALBNK 27MAR24APR SPD,1,1,FEDERALBNK25MAR25APRFUT,NaN,NaN
86509,NSEFO,10682542,4,WIPRO,WIPRO25FEB25MARFUT,FUTSTK,WIPRO-FUTSTK,2100110682542,7.80,-7.80,...,1,-1,NaN,2025-02-27T14:30:00,WIPRO 27FEB27MAR SPD,1,1,WIPRO25FEB25MARFUT,NaN,NaN
86510,NSEFO,10716410,4,JIOFIN,JIOFIN25MAR25APRFUT,FUTSTK,JIOFIN-FUTSTK,2100110716410,5.74,-5.74,...,1,-1,NaN,2025-03-27T14:30:00,JIOFIN 27MAR24APR SPD,1,1,JIOFIN25MAR25APRFUT,NaN,NaN


In [71]:
df5[df5["instrument_category"]==1]["underlying_name"].unique()
# Remove rows where underlying_name is NaN
df5 = df5.dropna(subset=["underlying_name"])
# Remove rows where underlying_name ends with TEST
df5 = df5[~df5["underlying_name"].str.endswith("TEST")]

In [72]:
df5[df5["instrument_category"]==1]["underlying_name"].unique()


array(['ASIANPAINT', 'JIOFIN', 'SBIN', 'WIPRO', 'M&M', 'DRREDDY',
       'CROMPTON', 'VBL', 'BAJAJ-AUTO', 'CIPLA', 'VEDL', 'PEL', 'IRB',
       'SONACOMS', 'HEROMOTOCO', 'DIVISLAB', 'ATGL', 'INDUSINDBK',
       'TATACHEM', 'TRENT', 'CYIENT', 'OFSS', 'MGL', 'LAURUSLABS',
       'TATAELXSI', 'Nifty Bank', 'AUROPHARMA', 'RBLBANK', 'JSWENERGY',
       'JINDALSTEL', 'BERGEPAINT', 'PHOENIXLTD', 'IDEA', 'TATAMOTORS',
       'BHARATFORG', 'MAXHEALTH', 'IRCTC', 'CHAMBLFERT', 'SUNPHARMA',
       'GODREJCP', 'ADANIENSOL', 'BALKRISIND', 'ASHOKLEY', 'IDFCFIRSTB',
       'PERSISTENT', 'ICICIGI', 'ULTRACEMCO', 'APOLLOTYRE', 'GODREJPROP',
       'BRITANNIA', 'BANDHANBNK', 'YESBANK', 'NATIONALUM', 'TATASTEEL',
       'SHREECEM', 'DABUR', 'CHOLAFIN', 'ICICIPRULI', 'BANKINDIA',
       'EICHERMOT', 'NMDC', 'ALKEM', 'AXISBANK', 'POLYCAB', 'ABFRL',
       'DIXON', 'Nifty Fin Service', 'NCC', 'INDUSTOWER', 'ADANIGREEN',
       'IPCALAB', 'SAIL', 'IOC', 'HINDALCO', 'NHPC', 'MANAPPURAM',
       'POLICYBZR', 'G

In [78]:
df5=df5[(df5["instrument_category"]==1) | (df5["instrument_category"]==4)]

In [79]:
df5["instrument_category"].unique()

array([1, 4])

In [80]:
df5["underlying_name"].unique()

array(['ASIANPAINT', 'JIOFIN', 'SBIN', 'WIPRO', 'M&M', 'DRREDDY',
       'CROMPTON', 'VBL', 'BAJAJ-AUTO', 'CIPLA', 'VEDL', 'PEL', 'IRB',
       'SONACOMS', 'HEROMOTOCO', 'DIVISLAB', 'ATGL', 'INDUSINDBK',
       'TATACHEM', 'TRENT', 'CYIENT', 'OFSS', 'MGL', 'LAURUSLABS',
       'TATAELXSI', 'Nifty Bank', 'AUROPHARMA', 'RBLBANK', 'JSWENERGY',
       'JINDALSTEL', 'BERGEPAINT', 'PHOENIXLTD', 'IDEA', 'TATAMOTORS',
       'BHARATFORG', 'MAXHEALTH', 'IRCTC', 'CHAMBLFERT', 'SUNPHARMA',
       'GODREJCP', 'ADANIENSOL', 'BALKRISIND', 'ASHOKLEY', 'IDFCFIRSTB',
       'PERSISTENT', 'ICICIGI', 'ULTRACEMCO', 'APOLLOTYRE', 'GODREJPROP',
       'BRITANNIA', 'BANDHANBNK', 'YESBANK', 'NATIONALUM', 'TATASTEEL',
       'SHREECEM', 'DABUR', 'CHOLAFIN', 'ICICIPRULI', 'BANKINDIA',
       'EICHERMOT', 'NMDC', 'ALKEM', 'AXISBANK', 'POLYCAB', 'ABFRL',
       'DIXON', 'Nifty Fin Service', 'NCC', 'INDUSTOWER', 'ADANIGREEN',
       'IPCALAB', 'SAIL', 'IOC', 'HINDALCO', 'NHPC', 'MANAPPURAM',
       'POLICYBZR', 'G

In [50]:
df5[df5["underlying"]=="TITAN"]

,exchange,token,instrument_category,underlying,contract_symbol,instrument_type,product,contract_unique_id,price1,price2,...,trading_status,alternate_id,underlying_name,expiry,strike,option_type,option_description,flag1,flag2,contract_code
85489,NSEFO,43400,1,TITAN,TITAN25MARFUT,FUTSTK,TITAN-FUTSTK,2508600043400,3613.45,2956.50,...,1,1100100003506,TITAN,2025-03-27T14:30:00,TITAN 27MAR2025,1,1,TITAN25MARFUT,NaN,NaN
85829,NSEFO,73632,1,TITAN,TITAN25APRFUT,FUTSTK,TITAN-FUTSTK,2511400073632,3636.35,2975.20,...,1,1100100003506,TITAN,2025-04-24T14:30:00,TITAN 24APR2025,1,1,TITAN25APRFUT,NaN,NaN
86025,NSEFO,41498,1,TITAN,TITAN25FEBFUT,FUTSTK,TITAN-FUTSTK,2505800041498,3594.25,2940.75,...,1,1100100003506,TITAN,2025-02-27T14:30:00,TITAN 27FEB2025,1,1,TITAN25FEBFUT,NaN,NaN
86166,NSEFO,10666888,4,TITAN,TITAN25FEB25MARFUT,FUTSTK,TITAN-FUTSTK,2100110666888,81.70,-81.70,...,1,-1,NaN,2025-02-27T14:30:00,TITAN 27FEB27MAR SPD,1,1,TITAN25FEB25MARFUT,NaN,NaN
86491,NSEFO,11184032,4,TITAN,TITAN25MAR25APRFUT,FUTSTK,TITAN-FUTSTK,2100111184032,82.15,-82.15,...,1,-1,NaN,2025-03-27T14:30:00,TITAN 27MAR24APR SPD,1,1,TITAN25MAR25APRFUT,NaN,NaN


In [ ]:
Nifty 50

In [34]:
df5

,exchange,token,instrument_category,underlying,contract_symbol,instrument_type,product,contract_unique_id,price1,price2,...,trading_status,alternate_id,underlying_name,expiry,strike,option_type,option_description,flag1,flag2,contract_code
85397,NSEFO,69079,1,ASIANPAINT,ASIANPAINT25APRFUT,FUTSTK,ASIANPAINT-FUTSTK,2511400069079,2485.15,2033.30,...,1,1100100000236,ASIANPAINT,2025-04-24T14:30:00,ASIANPAINT 24APR2025,1,1,ASIANPAINT25APRFUT,NaN,NaN
85398,NSEFO,73466,1,JIOFIN,JIOFIN25APRFUT,FUTSTK,JIOFIN-FUTSTK,2511400073466,254.49,208.22,...,1,1100100018143,JIOFIN,2025-04-24T14:30:00,JIOFIN 24APR2025,1,1,JIOFIN25APRFUT,NaN,NaN
85399,NSEFO,43355,1,SBIN,SBIN25MARFUT,FUTSTK,SBIN-FUTSTK,2508600043355,812.20,664.55,...,1,1100100003045,SBIN,2025-03-27T14:30:00,SBIN 27MAR2025,1,1,SBIN25MARFUT,NaN,NaN
85400,NSEFO,43438,1,WIPRO,WIPRO25MARFUT,FUTSTK,WIPRO-FUTSTK,2508600043438,343.70,281.25,...,1,1100100003787,WIPRO,2025-03-27T14:30:00,WIPRO 27MAR2025,1,1,WIPRO25MARFUT,NaN,NaN
85401,NSEFO,73487,1,M&M,M&M25APRFUT,FUTSTK,M&M-FUTSTK,2511400073487,3332.25,2726.40,...,1,1100100002031,M&M,2025-04-24T14:30:00,M&M 24APR2025,1,1,M&M25APRFUT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86072,NSEFO,42278,1,KPITTECH,KPITTECH25MARFUT,FUTSTK,KPITTECH-FUTSTK,2508600042278,1492.40,1221.05,...,1,1100100009683,KPITTECH,2025-03-27T14:30:00,KPITTECH 27MAR2025,1,1,KPITTECH25MARFUT,NaN,NaN
86073,NSEFO,40480,1,PAYTM,PAYTM25FEBFUT,FUTSTK,PAYTM-FUTSTK,2505800040480,823.55,673.80,...,1,1100100006705,PAYTM,2025-02-27T14:30:00,PAYTM 27FEB2025,1,1,PAYTM25FEBFUT,NaN,NaN
86074,NSEFO,69084,1,AXISBANK,AXISBANK25APRFUT,FUTSTK,AXISBANK-FUTSTK,2511400069084,1123.60,919.35,...,1,1100100005900,AXISBANK,2025-04-24T14:30:00,AXISBANK 24APR2025,1,1,AXISBANK25APRFUT,NaN,NaN
86075,NSEFO,43104,1,LAURUSLABS,LAURUSLABS25MARFUT,FUTSTK,LAURUSLABS-FUTSTK,2508600043104,666.05,544.95,...,1,1100100019234,LAURUSLABS,2025-03-27T14:30:00,LAURUSLABS 27MAR2025,1,1,LAURUSLABS25MARFUT,NaN,NaN


In [ ]:
df5[""].unique()

KeyError: 'OptionType'

In [21]:
futures=df5[df5["instrument_type"]=="FUTSTK"]

KeyError: 'instrument_type'

In [81]:
df5.rename(columns={"token":"ExchangeInstrumentID","contract_symbol":"Description"},inplace=True)

In [82]:
df5.rename(columns={"strike":"Strike"},inplace=True)

In [83]:
sortdf=df5.sort_values(by=["expiry"])

In [ ]:
import re
import json

def extract_futures(symbol, df):
    """
    Extracts future contract details from the given symbol.

    Args:
    symbol (str): The future contract description.
    df (DataFrame): The DataFrame containing futures contract data.

    Returns:
    tuple: (ExchangeInstrumentID of first expiry, ExchangeInstrumentID of second expiry)
    """
    # Regular expression to match stock symbol and expiry months
    pattern = re.match(r"([A-Z\&\-]+)(\d{2}[A-Z]+)(\d{2}[A-Z]+)FUT", symbol)
    
    if pattern:
        stock_symbol = pattern.group(1)  # Extract stock symbol (e.g., TITAN)
        expiry1 = pattern.group(2)       # First expiry (e.g., 25MAR)
        expiry2 = pattern.group(3)       # Second expiry (e.g., 25APR)

        # Fetch F1 ExchangeInstrumentID
        fut1_df = df[df['Description'] == f"{stock_symbol}{expiry1}FUT"]
        fut1_exchange_id = int(fut1_df['ExchangeInstrumentID'].values[0]) if not fut1_df.empty else None

        # Fetch F2 ExchangeInstrumentID
        fut2_df = df[df['Description'] == f"{stock_symbol}{expiry2}FUT"]
        fut2_exchange_id = int(fut2_df['ExchangeInstrumentID'].values[0]) if not fut2_df.empty else None

        return fut1_exchange_id, fut2_exchange_id
    else:
        return None, None

# Dictionary to store parent-child mapping
futures_mapping = {}

# List to store ExchangeInstrumentIDs
exchange_instrument_list = []

# Iterate over descriptions ending with "SPD"
for _, row in df5[df5["Strike"].str.endswith("SPD")].iterrows():
    parent_exchange_id = int(row['ExchangeInstrumentID'])  # Convert to int
    result = extract_futures(row['Description'], df5)

    if result:
        f1_exchange_id = result[0]
        f2_exchange_id = result[1]

        # Store only if both F1 and F2 are found
        if f1_exchange_id and f2_exchange_id:
            futures_mapping[parent_exchange_id] = [f1_exchange_id, f2_exchange_id]

            # Add all IDs to the list
            exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": f1_exchange_id})
            exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": f2_exchange_id})

    # Add parent exchangeInstrumentID to the list as well
    exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": parent_exchange_id})

# Remove duplicates from the list
exchange_instrument_list = list({v["exchangeInstrumentID"]: v for v in exchange_instrument_list}.values())

# Save Futures Mapping to JSON
futures_mapping_file = r"data\futures_mapping2.json"
with open(futures_mapping_file, "w") as json_file:
    json.dump(futures_mapping, json_file, indent=4)

# Save ExchangeInstrumentID List to JSON
exchange_instrument_file = r"data\exchange_instruments2.json"
with open(exchange_instrument_file, "w") as json_file:
    json.dump(exchange_instrument_list, json_file, indent=4)

print(f"✅ Futures mapping saved at: {futures_mapping_file}")
print(f"✅ Exchange Instrument List saved at: {exchange_instrument_file}")



#create a dic of ExchangeInstrumentID and  Description

df1=df5[["ExchangeInstrumentID","Description"]]
df1=df1.drop_duplicates()
df1=df1.set_index("ExchangeInstrumentID")
df1=df1.to_dict()
#save to json
import json
with open('ExchangeInstrumentID.json', 'w') as fp:
    json.dump(df1, fp)
    

✅ Futures mapping saved at: data\futures_mapping2.json
✅ Exchange Instrument List saved at: data\exchange_instruments2.json


In [61]:
df1=df5[["ExchangeInstrumentID","Description"]]
df1=df1.drop_duplicates()
df1=df1.set_index("ExchangeInstrumentID")
df1=df1.to_dict()
#save to json
import json
with open('ExchangeInstrumentID2.json', 'w') as fp:
    json.dump(df1, fp)
    

In [84]:
df5["lot_size"]

85397      200
85398     1650
85399      750
85400     3000
85401      175
         ...  
86507    10250
86508     5000
86509     3000
86510     1650
86511     1700
Name: lot_size, Length: 1115, dtype: int64

In [85]:
df6=df5.copy

In [86]:
df1 = df5[["ExchangeInstrumentID", "lot_size"]]
df1 = df1.drop_duplicates()
df1 = df1.set_index("ExchangeInstrumentID")
df1 = df1.to_dict()
# save to json
import json
with open('lotsize.json', 'w') as fp:
    json.dump(df1, fp)
    

In [62]:
futures["lot_size"]

85397      200
85398     1650
85399      750
85400     3000
85401      175
         ...  
86507    10250
86508     5000
86509     3000
86510     1650
86511     1700
Name: lot_size, Length: 1085, dtype: int64